In [6]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [7]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql.window import Window
import time
import shutil
from pyspark.sql.functions import current_timestamp, col, unix_timestamp

from datetime import datetime, timedelta


import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


In [8]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

In [9]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import subprocess
# import os
# from pyspark.sql import functions as F
# from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType

# # Schema của dữ liệu Kafka
# schema = StructType([
#     StructField("timestamp", StringType()),
#     StructField("prices", StructType([
#         StructField("bitcoin", FloatType()),
#         StructField("ethereum", FloatType()),
#         StructField("tether", FloatType()),
#         StructField("usd-coin", FloatType()),
#         StructField("ripple", FloatType()),
#         StructField("cardano", FloatType()),
#         StructField("dogecoin", FloatType()),
#         StructField("matic-network", FloatType()),
#         StructField("solana", FloatType()),
#         StructField("litecoin", FloatType()),
#         StructField("polkadot", FloatType()),
#         StructField("shiba-inu", FloatType()),
#         StructField("tron", FloatType()),
#         StructField("cosmos", FloatType()),
#         StructField("chainlink", FloatType()),
#         StructField("stellar", FloatType()),
#         StructField("near", FloatType()),
#     ]))
# ])

# # Đọc dữ liệu từ Kafka
# kafka_df = spark.readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
#     .option("subscribe", "crypto-pricessss") \
#     .option("startingOffsets", "latest") \
#     .option("maxOffsetsPerTrigger", 1000) \
#     .load()

# parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
#     .select(F.from_json(F.col("value"), schema).alias("data"))

# crypto_parsed_df = parsed_df.select(
#     F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
#     F.col("data.prices.bitcoin").alias("BTC"),
#     F.col("data.prices.ethereum").alias("ETH"),
#     F.col("data.prices.tether").alias("USDT"),
#     F.col("data.prices.usd-coin").alias("USDC"),
#     F.col("data.prices.ripple").alias("XRP"),
#     F.col("data.prices.cardano").alias("ADA"),
#     F.col("data.prices.dogecoin").alias("DOGE"),
#     F.col("data.prices.matic-network").alias("MATIC"),
#     F.col("data.prices.solana").alias("SOL")
# )

# column_names = [col for col in crypto_parsed_df.columns if col != 'DATE']

# historical_schema = StructType([
#     StructField("BASE", StringType(), True),  
#     StructField("DATE", StringType(), True),
#     StructField("OPEN", DoubleType(), True),
#     StructField("HIGH", DoubleType(), True),
#     StructField("LOW", DoubleType(), True),
#     StructField("CLOSE", DoubleType(), True),
#     StructField("VOLUME", DoubleType(), True),
#     StructField("YEAR", IntegerType(), True),
#     StructField("MONTH", IntegerType(), True),
#     StructField("__index_level_0__", LongType(), True)
# ])


# def read_historical_data(coin):
#     return (
#         spark.read.schema(historical_schema).format("parquet")
#         .load(f"gs://crypto-historical-data-2/ver2/{coin}/2024/*")
#         .select(F.col("DATE").cast("timestamp"), "CLOSE")
#     )

# def process_coin(coin, micro_batch_latest_df):
#     historical_data_df = read_historical_data(coin)
#     micro_batch = micro_batch_latest_df.select("DATE", coin).withColumnRenamed(coin, "CLOSE")
#     combined_df = micro_batch.unionByName(historical_data_df)

#     window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
#     combined_df = combined_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
#     combined_df = combined_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
#     combined_df = combined_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
#     combined_df = combined_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
#     combined_df = combined_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
#     combined_df = combined_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

#     combined_df = combined_df.orderBy("DATE", ascending=False)
    
#     current_date = F.current_date() 
#     combined_df = combined_df.filter(F.col("DATE") >= current_date)

#     tmp_dir = f"gs://indicator-crypto/sma_results/tmp/{coin}"

#     #combined_df.write.format("console").option("truncate", False).save()
#     combined_df.write \
#         .format("csv") \
#         .option("header", "true") \
#         .option("path", tmp_dir) \
#         .mode("append") \
#         .save()
    
# def process_batch(micro_batch_df, batch_id):
#     micro_batch_latest_df = (
#         micro_batch_df
#         .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
#         .filter(F.col("row_num") == 1)
#         .drop("row_num")
#     )
    
#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(process_coin, coin, micro_batch_latest_df) for coin in column_names]
#         for future in as_completed(futures):
#             print(future.result()) 

# # Thực thi stream
# query = crypto_parsed_df.writeStream \
#     .foreachBatch(process_batch) \
#     .start()

# query.awaitTermination()


In [10]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from concurrent.futures import ThreadPoolExecutor, as_completed
import subprocess
import os
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType

# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()

parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("BTC"),
    F.col("data.prices.ethereum").alias("ETH"),
    F.col("data.prices.tether").alias("USDT"),
    F.col("data.prices.usd-coin").alias("USDC"),
    F.col("data.prices.ripple").alias("XRP"),
    F.col("data.prices.cardano").alias("ADA"),
    F.col("data.prices.dogecoin").alias("DOGE"),
    F.col("data.prices.matic-network").alias("MATIC"),
    F.col("data.prices.solana").alias("SOL")
)

column_names = [col for col in crypto_parsed_df.columns if col != 'DATE']

historical_schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])


def read_historical_data(coin):
    return (
        spark.read.schema(historical_schema).format("parquet")
        .load(f"gs://crypto-historical-data-2/ver2/{coin}/2024/*")
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )
def calculate_rsi(df, window_size=14):
    # Tính sự thay đổi của giá
    df = df.withColumn("price_change", F.col("CLOSE") - F.lag("CLOSE").over(Window.orderBy("DATE")))
    
    # Tính Gain và Loss
    df = df.withColumn("gain", F.when(F.col("price_change") > 0, F.col("price_change")).otherwise(0))
    df = df.withColumn("loss", F.when(F.col("price_change") < 0, -F.col("price_change")).otherwise(0))
    
    # Tính Average Gain và Average Loss
    window_spec = Window.orderBy("DATE").rowsBetween(-window_size + 1, 0)
    
    df = df.withColumn("avg_gain", F.avg("gain").over(window_spec))
    df = df.withColumn("avg_loss", F.avg("loss").over(window_spec))
    
    # Tính Relative Strength (RS)
    df = df.withColumn("rs", F.col("avg_gain") / F.col("avg_loss"))
    
    # Tính RSI
    df = df.withColumn("RSI_14", 100 - (100 / (1 + F.col("rs"))))
    
    return df

def process_coin(coin, micro_batch_latest_df):
    historical_data_df = read_historical_data(coin)
    micro_batch = micro_batch_latest_df.select("DATE", coin).withColumnRenamed(coin, "CLOSE")
    combined_df = micro_batch.unionByName(historical_data_df)

    # Tính RSI-14
    combined_df = calculate_rsi(combined_df)

    # Sắp xếp dữ liệu theo ngày
    combined_df = combined_df.orderBy("DATE", ascending=False)
    
    current_date = F.current_date() 
    combined_df = combined_df.filter(F.col("DATE") >= current_date)

    tmp_dir = f"gs://indicator-crypto/rsi_results/tmp/{coin}"
    combined_df.write.format("console").option("truncate", False).save()
    # combined_df.write \
    #     .format("csv") \
    #     .option("header", "true") \
    #     .option("path", tmp_dir) \
    #     .mode("append") \
    #     .save()
    
def process_batch(micro_batch_df, batch_id):
    micro_batch_latest_df = (
        micro_batch_df
        .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
        .filter(F.col("row_num") == 1)
        .drop("row_num")
    )
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_coin, coin, micro_batch_latest_df) for coin in column_names]
        for future in as_completed(futures):
            print(future.result()) 

# Thực thi stream
query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()


24/12/14 16:20:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/temporary-bf953da0-4e27-4f56-8722-54b686d32abc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/14 16:20:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/14 16:20:09 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/12/14 16:20:09 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/12/14 16:20:09 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/12/14 16:20:09 WARN AdminClientConfig: The configuration '

+----+-----+------------+----+----+--------+--------+---+------+
|DATE|CLOSE|price_change|gain|loss|avg_gain|avg_loss|rs |RSI_14|
+----+-----+------------+----+----+--------+--------+---+------+
+----+-----+------------+----+----+--------+--------+---+------+



+----+-----+------------+----+----+--------+--------+---+------+
|DATE|CLOSE|price_change|gain|loss|avg_gain|avg_loss|rs |RSI_14|
+----+-----+------------+----+----+--------+--------+---+------+
+----+-----+------------+----+----+--------+--------+---+------+



+----+-----+------------+----+----+--------+--------+---+------+
|DATE|CLOSE|price_change|gain|loss|avg_gain|avg_loss|rs |RSI_14|
+----+-----+------------+----+----+--------+--------+---+------+
+----+-----+------------+----+----+--------+--------+---+------+



+----+-----+------------+----+----+--------+--------+---+------+
|DATE|CLOSE|price_change|gain|loss|avg_gain|avg_loss|rs |RSI_14|
+----+-----+------------+----+----+--------+--------+---+------+
+----+-----+------------+----+----+--------+--------+---+------+

+----+-----+------------+----+----+--------+--------+---+------+
|DATE|CLOSE|price_change|gain|loss|avg_gain|avg_loss|rs |RSI_14|
+----+-----+------------+----+----+--------+--------+---+------+
+----+-----+------------+----+----+--------+--------+---+------+



...

+--------------------------+--------+-----------------+-----------------+----+-----------------+-----------------+------------------+------------------+
|DATE                      |CLOSE   |price_change     |gain             |loss|avg_gain         |avg_loss         |rs                |RSI_14            |
+--------------------------+--------+-----------------+-----------------+----+-----------------+-----------------+------------------+------------------+
|2024-12-14 16:24:27.920937|101644.0|1746.199999999997|1746.199999999997|0.0 |946.7571428571422|650.0828571428564|1.4563638041744136|59.289418029179075|
+--------------------------+--------+-----------------+-----------------+----+-----------------+-----------------+------------------+------------------+



+--------------------------+------------------+-------------------+-------------------+----+-------------------+-------------------+-----------------+-----------------+
|DATE                      |CLOSE             |price_change       |gain               |loss|avg_gain           |avg_loss           |rs               |RSI_14           |
+--------------------------+------------------+-------------------+-------------------+----+-------------------+-------------------+-----------------+-----------------+
|2024-12-14 16:24:27.920937|2.4600000381469727|0.06096003814697282|0.06096003814697282|0.0 |0.11348714558192663|0.06650142857142859|1.706536957473494|63.05241658575086|
+--------------------------+------------------+-------------------+-------------------+----+-------------------+-------------------+-----------------+-----------------+



+--------------------------+------------------+--------------------+--------------------+----+------------------+--------------------+------------------+------------------+
|DATE                      |CLOSE             |price_change        |gain                |loss|avg_gain          |avg_loss            |rs                |RSI_14            |
+--------------------------+------------------+--------------------+--------------------+----+------------------+--------------------+------------------+------------------+
|2024-12-14 16:24:27.920937|0.6176900267601013|0.006290026760101264|0.006290026760101264|0.0 |0.0158850019114358|0.013442857142857137|1.1816685800217923|54.163523774540906|
+--------------------------+------------------+--------------------+--------------------+----+------------------+--------------------+------------------+------------------+

+--------------------------+-------------------+--------------------+--------------------+----+--------------------+------------------

+--------------------------+------------------+--------------------+--------------------+----+--------------------+-------------------+------------------+------------------+
|DATE                      |CLOSE             |price_change        |gain                |loss|avg_gain            |avg_loss           |rs                |RSI_14            |
+--------------------------+------------------+--------------------+--------------------+----+--------------------+-------------------+------------------+------------------+
|2024-12-14 16:24:27.920937|1.1100000143051147|0.025274014305114667|0.025274014305114667|0.0 |0.024157643878936756|0.02173621428571428|1.1114007048970767|52.638075866856994|
+--------------------------+------------------+--------------------+--------------------+----+--------------------+-------------------+------------------+------------------+

+--------------------------+------------------+----------------------+----+---------------------+---------------------+----------

+--------------------------+------------------+----------------------+----+---------------------+---------------------+---------------------+------------------+------------------+
|DATE                      |CLOSE             |price_change          |gain|loss                 |avg_gain             |avg_loss             |rs                |RSI_14            |
+--------------------------+------------------+----------------------+----+---------------------+---------------------+---------------------+------------------+------------------+
|2024-12-14 16:24:27.920937|0.9998670220375061|-1.3297796249389648E-4|0.0 |1.3297796249389648E-4|2.8571428571425423E-5|3.0926997320990246E-5|0.9238345473659646|48.020478093131295|
+--------------------------+------------------+----------------------+----+---------------------+---------------------+---------------------+------------------+------------------+

+--------------------------+------------------+--------------------+--------------------+----+-----